### Ingestion del archivo "language_role.json"

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

#### Paso 1 - Leer el archivo JSON utilizando "DataFrameReader" de Spark

In [0]:
language_role_schema = "roleId INT, languageRole STRING"

In [0]:
language_role_df = spark.read \
    .schema(language_role_schema) \
    .json(f"{bronze_folder_path}/{v_file_date}/language_role.json")

#### Paso 3 - Cambiar el nombre de las columnas segun lo "requerido"

In [0]:
language_role_renamed_df = language_role_df \
    .withColumnRenamed("roleId","role_id") \
    .withColumnRenamed("languageRole","language_role") 

#### Paso 3 - Agregar una columna "ingestion date" al dataframe

In [0]:
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import lit

In [0]:
language_role_final_df = add_ingestion_date(language_role_renamed_df) \
.withColumn("enviroment", lit(dbutils.widgets.get("p_enviroment")))\
.withColumn("file_date", lit(v_file_date))

#### Paso 4 - Escribir datos en Datalake en formato parquet

In [0]:
#language_role_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/language_role")
language_role_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.language_role");

In [0]:
%sql
SELECT * FROM movie_silver.language_role